In [11]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.optimizers import RMSprop
from cvzone.HandTrackingModule import HandDetector
from cvzone.PoseModule import PoseDetector
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)
train_dataset = train.flow_from_directory("BaseData/Training", 
                                          target_size=(300,300),
                                          batch_size = 50,
                                          class_mode='categorical')

validation_dataset = train.flow_from_directory("BaseData/Validation", 
                                          target_size=(300,300),
                                          batch_size = 50,
                                          class_mode='categorical')

Found 93622 images belonging to 28 classes.
Found 5872 images belonging to 28 classes.


In [13]:
train_dataset.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'Delete': 4,
 'E': 5,
 'F': 6,
 'G': 7,
 'H': 8,
 'I': 9,
 'J': 10,
 'K': 11,
 'L': 12,
 'M': 13,
 'N': 14,
 'O': 15,
 'P': 16,
 'Q': 17,
 'R': 18,
 'S': 19,
 'Space': 20,
 'T': 21,
 'U': 22,
 'V': 23,
 'W': 24,
 'X': 25,
 'Y': 26,
 'Z': 27}

In [14]:
input_img = tf.keras.layers.Input(shape =(300,300,3))

In [15]:
input_img = tf.keras.layers.Input(shape =(300,300,3))
model = tf.keras.models.Sequential([input_img, tf.keras.layers.Conv2D(16,(3,3),activation = "relu"),
                                   tf.keras.layers.MaxPool2D(2,2),
                                    #
                                   tf.keras.layers.Conv2D(32,(3,3),activation = "relu"),
                                   tf.keras.layers.MaxPool2D(2,2),
                                    #
                                   tf.keras.layers.Conv2D(64,(3,3),activation = "relu"),
                                   tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                   tf.keras.layers.Flatten(),
                                    ##
                                    tf.keras.layers.Dense(512,activation="relu"),
                                    ##
                                    tf.keras.layers.Dense(28,activation="softmax"),
                                   ])

In [16]:
optimizer = RMSprop(learning_rate=0.001)
model.compile(loss="categorical_crossentropy",
             optimizer = optimizer,
             metrics = ['accuracy'])

In [17]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 298, 298, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 149, 149, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 147, 147, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 73, 73, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 71, 71, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 35, 35, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 78400)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │      40,141,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 28)                  │          14,364 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,179,260 (153.27 MB)

 Trainable params: 40,179,260 (153.27 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

model_fit = model.fit(train_dataset,
                      steps_per_epoch = 200,
                      epochs = 10,
                      validation_data = validation_dataset,
                      callbacks=[early_stopping])
                      

Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 2233s 10s/step - accuracy: 0.1078 - loss: 3.4059 - val_accuracy: 0.2990 - val_loss: 2.5985
Epoch 2/10
194/200 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.5192 - loss: 1.5932

In [10]:
model.save("my_model_two.keras")

In [ ]:
camera = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
spacing = 20
imgWhiteSize = 300

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        imgWhite = np.ones((imgWhiteSize, imgWhiteSize, 3), np.uint8) * 250
        imgCrop = img[y-spacing:y + h + spacing, x - spacing:x + w + spacing]
        imgCropShape = imgCrop.shape


        aspectRatio = h/w

        if aspectRatio > 1:
            k = imgWhiteSize / h
            wCalculated = math.ceil(k*w)
            imgResize = cv2.resize(imgCrop, (wCalculated,imgWhiteSize))
            imgResizeShape = imgResize.shape
            widthGap = math.ceil((imgWhiteSize-wCalculated)/2)
            imgWhite[:, widthGap:wCalculated+widthGap] = imgResize

        else:
            k = imgWhiteSize / w
            hCalculated = math.ceil(k*h)
            imgResize = cv2.resize(imgCrop, (imgWhiteSize,hCalculated))
            imgResizeShape = imgResize.shape
            heightGap = math.ceil((imgWhiteSize-hCalculated)/2)
            imgWhite[heightGap:hCalculated + heightGap, :] = imgResize

        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImgWhite", imgWhite)

    cv2.imshow("Image", img)
    key = cv2.waitKey(1)